In [ ]:
import edrixs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import ListedColormap

from optEDRIXS import run_bayesian_optimization, import_config, reconstBOresults, getMask

import scipy
import json
from pathlib import Path

In [ ]:
def turbo_w():
    """
    modified turbo colormap with white background
    cmap = turbo_w()
    """
    cmap = np.load('/home/raynor/Dropbox/RIXS/dismeas/white_turbo.npy')
    return ListedColormap(cmap)

In [ ]:
cfg = import_config("NiPS3_config.py")

In [ ]:
rixs_ref = cfg.rixs_ref

In [ ]:
record = cfg.record
params_dict = reconstBOresults(record)
tAll0 = params_dict['tAll']
paramsAll0= params_dict['paramsAll']

params=paramsAll0.T.reshape(7,60,1010)
t=tAll0.reshape(60,1010)

In [ ]:
record["true_values"]

In [ ]:
record["pbounds"]

In [ ]:
paramsAll0.shape

In [ ]:
priorGuesses=record['true_values']

In [ ]:
numRuns=record['num_runs_local']*record['num_runs_global']
numIters=record['num_iters']
numParams=len(record['pbounds'])

In [ ]:
parSoFar = np.zeros([numParams,numRuns,numIters])
tSoFar = np.zeros([numRuns,numIters])
cur = np.array(t[:,0])
parCur = np.zeros([numParams,numRuns])
for i in range(numParams):
    parCur[i] = params[i,:,0]
    
for j in range(numIters):
    for i in range(numRuns):
        if (t[i][j] > cur[i]):
            cur[i] = t[i][j]
            parCur[:,i] = params[:,i,j]
    parSoFar[:,:,j] = parCur
    tSoFar[:,j] = cur
labels=params_dict["labels"]

In [ ]:
labels

In [ ]:
# RevTeX single‐column width is ≈3.4 in; height up to ≈2.5 in
fig_w, fig_h = 3.4, 2.5  # inches

# -- Apply style overrides --------------------------------------
plt.rcParams.update({
    "figure.figsize": (fig_w, fig_h),
    "figure.dpi": 300,
    "font.size": 9,                 # base size for text
    "axes.labelsize": 9,            # x/y label size
    "axes.titlesize": 9,            # if you use titles
    "xtick.labelsize": 9,            # tick labels slightly smaller
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "legend.frameon": False,
    "axes.linewidth": 0.8,
})

# -- Plot --------------------------------------------------------
xpts = np.arange(1, numIters + 1)
fig, ax = plt.subplots()
ax.set_xlabel('Iteration')
ax.set_ylabel('-Target')

for i in range(numRuns):
    ax.plot(xpts, -tSoFar[i], lw=1)

# -- Layout tweaks ----------------------------------------------
plt.tight_layout(pad=0.2)  # minimal padding to fit labels cleanly
# or, if you need manual control:
# plt.subplots_adjust(left=0.18, bottom=0.18, right=0.98, top=0.97)

# -- Save or show -----------------------------------------------
plt.savefig('NiPS3_target.pdf',bbox_inches="tight", pad_inches=0.02)

In [ ]:
paramsAll = paramsAll0[-tAll0 < np.min(-tAll0)*record['threshold_factor']]
tAll = tAll0[-tAll0 < np.min(-tAll0)*record['threshold_factor']]

len_thresh = len(tAll)

filters = record['greedy_filters']

param_names = params_dict['labels']
col = { name: paramsAll[:, i]
        for i, name in enumerate(param_names) }

mask_p = getMask(len_thresh, filters, col)

plistSelect = paramsAll[mask_p]
tAllSelect = tAll[mask_p]

In [ ]:
paramsAll.shape

In [ ]:
pAllF2=paramsAll[:,0];
pAllF4=paramsAll[:,2];
pAll10Dq=paramsAll[:,5];

In [ ]:
#fig, ax = plt.subplots(nrows=1, ncols=1, dpi=300,figsize=(3+3/8, 2.8))
# RevTeX single‐column width is ≈3.4 in; height up to ≈2.5 in
fig_w, fig_h = 3.4, 2.8  # inches

# -- Apply style overrides --------------------------------------
plt.rcParams.update({
    "figure.figsize": (fig_w, fig_h),
    "figure.dpi": 300,
    "font.size": 9,                 # base size for text
    "axes.labelsize": 9,            # x/y label size
    "axes.titlesize": 9,            # if you use titles
    "xtick.labelsize": 9,            # tick labels slightly smaller
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "legend.frameon": False,
    "axes.linewidth": 0.8,
})
fig, ax = plt.subplots()

scatplot=ax.scatter(pAllF4,pAllF2,c=pAll10Dq,s=10,vmin=np.min(pAll10Dq), vmax=np.max(pAll10Dq),cmap=turbo_w())
ax.plot([0,9.6276],[0,9.6276])
ax.plot([0*0.625,9.6276*0.625],[0,9.6276])
ax.set_xlim([0,9.6276])
ax.set_ylim([0,9.6276])
ax.set_xlabel(r"$F^4_{dd}\:(\text{eV})$")
ax.set_ylabel(r"$F^2_{dd}\:(\text{eV})$")
plt.subplots_adjust(left=0.2,bottom=0.2)
fig.colorbar(scatplot,ax=ax,orientation='vertical',label=r'$10Dq\:(\text{eV})$')

plt.tight_layout(pad=0.2) 

plt.savefig("NiPS3_scatter1.pdf",bbox_inches="tight", pad_inches=0.02)

In [ ]:
coordsfin=np.loadtxt(cfg.data_str+cfg.data_opt+".txt")

In [ ]:
path = Path("NiPS3_clusters.json")  # change to your file
with path.open("r", encoding="utf-8") as f:
    cluster_data: dict = json.load(f)

In [ ]:
valtab=np.array(cluster_data['valtab'])
disttab=np.array(cluster_data['disttab'])
clusterinds=cluster_data['clusterinds']

In [ ]:
fig_w, fig_h = 3.4, 2.8  # inches

# -- Apply style overrides --------------------------------------
plt.rcParams.update({
    "figure.figsize": (fig_w, fig_h),
    "figure.dpi": 300,
    "font.size": 9,                 # base size for text
    "axes.labelsize": 9,            # x/y label size
    "axes.titlesize": 9,            # if you use titles
    "xtick.labelsize": 9,            # tick labels slightly smaller
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "legend.frameon": False,
    "axes.linewidth": 0.8,
})
fig, ax = plt.subplots()

scatplot=ax.scatter(coordsfin[:,2],coordsfin[:,0],c=coordsfin[:,5],s=10,vmin=1.00, vmax=1.15,cmap=turbo_w())
ax.plot([0,9.6276],[0,9.6276])
ax.plot([0*0.625,9.6276*0.625],[0,9.6276])
ax.scatter([coordsfin[clusterinds[0][np.argmin(valtab[clusterinds[0]])]][2]],[coordsfin[clusterinds[0][np.argmin(valtab[clusterinds[0]])]][0]],
        color="black",marker='x',s=100,alpha=0.5)
#ax.plot([coordsfin[clusterinds[0][np.argmin(valtab[clusterinds[0]])]][2]],[coordsfin[clusterinds[0][np.argmin(valtab[clusterinds[0]])]][0]],marker='*',color='black',markersize=10)

ax.set_xlim([0,9.6276])
ax.set_ylim([0,9.6276])
#for i in range(len(coordsfin)):
#    ax.plot([plistSelectInds[i,2],coordsfin[i,2]],[plistSelectInds[i,0],coordsfin[i,0]],c=plt.cm.jet(coordsfin[i,2]/10))
ax.set_xlabel(r"$F^4_{dd}\:(\text{eV})$")
ax.set_ylabel(r"$F^2_{dd}\:(\text{eV})$")
plt.subplots_adjust(left=0.2,bottom=0.2)
fig.colorbar(scatplot,ax=ax,orientation='vertical',label=r'$10Dq\:(\text{eV})$')

plt.tight_layout(pad=0.2) 

plt.savefig("NiPS3_scatter2.pdf",bbox_inches="tight", pad_inches=0.02)

In [ ]:
labels0=[k for k in cfg.greedy_bounds.keys()]

This is the final result after the greedy postprocessing (copy the output of the line below into testparam_NiPS3.json): 

In [ ]:
result = {labels0[i]: coordsfin[np.argmin(valtab)][i] for i in range(len(labels0))}
specres = cfg.rixs_funV({**result,**(cfg.fixed_params_greedy)})
result

This is the reference:

In [ ]:
startref=np.array([5.26,6.18,3.29,2.89,1.65,1.07,0,0.083,0.102,11.2,0.6])
startref[6]=-0.928 # This has been set manually to fix the horizontal shift of peaks
result_ref = {labels0[i]: startref[i] for i in range(len(labels0))}
specstart = cfg.rixs_funV({**result_ref,**(cfg.fixed_params_greedy)})
result_ref

This is the outcome of GPR before BOBYQA:

In [ ]:
GPRnums=np.concatenate([plistSelect[np.argmin(-tAllSelect)],[priorGuesses['soc_v_i'],priorGuesses['soc_v_n'],priorGuesses['soc_c'],priorGuesses['Gam_c']]])
GPRresult = {labels0[i]: GPRnums[i] for i in range(len(labels0))}
specGPR = cfg.rixs_funV({**GPRresult,**(cfg.fixed_params_greedy)})
GPRresult

In [ ]:
# will be erased:
ombounds = [0.,9.0]
omref= 40

eloss = np.linspace(0.5,2.0,151)

In [ ]:
plt.rcParams.update({
    "figure.dpi": 300,
    "font.size":       9,   # base text
    "axes.labelsize":  9,   # x/y labels
    "axes.titlesize":  9,   # if you ever title subplots
    "xtick.labelsize": 9,    # a bit smaller
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "axes.linewidth": 0.8,
    "figure.figsize":  (3.4, 4.0),  # single-column width ≈3.4 in, height chosen by you
})

fig = plt.figure()

gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1,1,1])
ax0 = plt.subplot(gs[0])
ax0.set_xlim(ombounds[0],ombounds[1])
ax0.set_xticks([0,3,6,9])
ax0.set_ylim(np.min(eloss), np.max(eloss))
ax1 = plt.subplot(gs[1],sharey = ax0)
ax2 = plt.subplot(gs[2],sharey = ax1)
ax3 = plt.subplot(gs[3],sharey = ax2)

ax1.set_xticks([0,3,6,9])
ax2.set_xticks([0,3,6,9])
ax3.set_xticks([0,3,6,9])
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax3.get_yticklabels(), visible=False)
ax0.text(0.1,2.05,'a)')
ax1.text(0.1,2.05,'b)')
ax2.text(0.1,2.05,'c)')
ax3.text(0.1,2.05,'d)')

ax0.imshow(rixs_ref/np.max(rixs_ref),  interpolation=None, aspect='auto', origin='lower', cmap=turbo_w(), rasterized=True,
                            extent=[ombounds[0], ombounds[1], np.min(eloss), np.max(eloss)])
subplot1=ax1.imshow(specstart/np.max(specstart), interpolation=None, aspect='auto', origin='lower',  cmap=turbo_w(), rasterized=True,
                           extent=[ombounds[0], ombounds[1], np.min(eloss), np.max(eloss)])
ax2.imshow(specGPR/np.max(specGPR), interpolation=None, aspect='auto', origin='lower',  cmap=turbo_w(), rasterized=True,
                           extent=[ombounds[0], ombounds[1], np.min(eloss), np.max(eloss)])
ax3.imshow(specres/np.max(specres), interpolation=None, aspect='auto', origin='lower',  cmap=turbo_w(), rasterized=True,
                           extent=[ombounds[0], ombounds[1], np.min(eloss), np.max(eloss)])

fig.subplots_adjust(left=0.2,bottom=0.25)
#cbar_ax=fig.add_axes([0.85,0.15,0.05,0.7])

cbar_ax=fig.add_axes([0.05,0.05,0.85,0.05])
fig.colorbar(subplot1,cax=cbar_ax,location='bottom',label="Intensity (arb. units)")

fig.supxlabel(r"$\Delta\omega_{\rm in}\:(\text{eV})$",
              x=0.52,    # centered
              y=0.15,   # just above the colorbar
              fontsize=9)
#ax0.set_xlabel(r"$\Delta\omega_{in}/eV$")
ax0.set_ylabel(r"$E_\text{loss}\:(\text{eV})$")
#ax1.set_xlabel(r"$\Delta\omega_{in}/eV$")
#ax2.set_xlabel(r"$\Delta\omega_{in}/eV$")
#ax3.set_xlabel(r"$\Delta\omega_{in}/eV$")
plt.subplots_adjust(wspace=.0)

plt.savefig("NiPS3_spectra.pdf",bbox_inches="tight", pad_inches=0.02)

In [ ]:
bnds0=[[3.0585, 14.6808],
 [0.0, 15.0],
 [1.8995, 9.1176],
 [-3.0, 6.0],
 [0.0, 4.0],
 [0.5, 4],
 [-12, 12],
 [0, 0.2],
 [0, 0.2],
 [6.0, 15.0],
 [0.1, 0.8]];

In [ ]:
# Calculate 1D slices around chosen solution. This can take a while (~10s of minutes)

In [ ]:
plotEstimates=[]

for ind00 in range(len(labels0)):
    print(ind00)
    plotParams=[]
    plotEstimates1=[]
    for i in range(100):
        plotParams.append(coordsfin[np.argmin(valtab)])
    plotParams=np.array(plotParams)
    plotParams[:,ind00]=np.linspace(bnds0[ind00][0],bnds0[ind00][1],100)
    for i in range(100):
        plotEstimates1.append(cfg.funAll(*plotParams[i]))
    plotEstimates.append(plotEstimates1)

In [ ]:
plotParams[0]

In [ ]:
labelsFig=['$F^2_{dd}$','$F^2_{dp}$','$F^4_{dd}$','$G^1_{dp}$','$G^3_{dp}$','$10Dq$','$\Delta\omega_{in}$','$\zeta_{i}$','$\zeta_{n}$','$\zeta_{c}$','$\Gamma_n$']

In [ ]:
optvec=cfg.funAll(*(coordsfin[np.argmin(valtab)]))

In [ ]:
plotEstimates=np.array(plotEstimates)

In [ ]:
plt.figure(dpi=150,figsize=(8.,8.))
plt.subplots_adjust(hspace=0.5,wspace=0.5)
plotParams=np.zeros([100,len(labels0)])
for ind in range(len(labels0)):
    plotParams[:,ind]=np.linspace(bnds0[ind][0],bnds0[ind][1],100)
    plt.subplot(4,3,ind+1)
    plt.xlabel(labelsFig[ind])
    plt.ylabel('$\chi^2_{L_1}/\chi^2_{L_1,min}$')
   # plt.title(labels0[ind])
    plt.xlim(bnds0[ind])

    plt.plot(plotParams[:,ind],plotEstimates[ind][:,1]/optvec[1],'--')
    plt.plot(plotParams[:,ind],plotEstimates[ind][:,2]/optvec[2],'--')
    plt.plot(plotParams[:,ind],plotEstimates[ind][:,3]/optvec[3],'--')
    plt.plot(plotParams[:,ind],plotEstimates[ind][:,0]/optvec[0])
    plt.axvline([coordsfin[np.argmin(valtab)][ind]],linestyle='solid',color='0.8')
    plt.plot([coordsfin[np.argmin(valtab)][ind]],1,'x',c='r')
plt.savefig("NiPS3_distances.pdf",bbox_inches="tight",   # include everything
    pad_inches=0.1)

In [ ]:
plotEstimates.shape